In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

Using CNTK backend


In [2]:
import sgan

In [3]:
def build_generator():

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=100))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(1, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(100,))
    img = model(noise)

    return Model(noise, img)

In [4]:
def build_discriminator(num_classes):

    img_shape = (28,28,1)

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.summary()

    img = Input(shape=img_shape)

    features = model(img)
    valid = Dense(1, activation="sigmoid")(features)
    label = Dense(num_classes+1, activation="softmax")(features)

    return Model(img, [valid, label])

In [5]:
num_classes = 10
descriminator = build_discriminator(num_classes)
generator = build_generator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

In [6]:
test_gan = sgan.SGAN(descriminator,generator,num_classes)
test_gan.build_networks()

In [7]:
num_classes = 10
(X_train, y_train), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)
y_train = y_train.reshape(-1, 1)

In [18]:
test_gan.train(X_train, y_train, epochs=2000, save_interval=50)

G:\Anaconda2\envs\tensorflow\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input2107") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
G:\Anaconda2\envs\tensorflow\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input2931") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
G:\Anaconda2\envs\tensorflow\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input2933") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
G:\Anaconda2\envs\tensorflow\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (u

0 [D loss: 0.413685, acc: 67.97%, op_acc: 44.53%] [G loss: 1.037553]
1 [D loss: 0.418315, acc: 69.53%, op_acc: 43.75%] [G loss: 1.113632]
2 [D loss: 0.408312, acc: 71.88%, op_acc: 48.44%] [G loss: 1.113013]
3 [D loss: 0.408167, acc: 70.31%, op_acc: 44.53%] [G loss: 1.074810]
4 [D loss: 0.419509, acc: 67.19%, op_acc: 45.31%] [G loss: 1.044287]
5 [D loss: 0.432668, acc: 66.41%, op_acc: 44.53%] [G loss: 0.927006]
6 [D loss: 0.434101, acc: 64.06%, op_acc: 46.09%] [G loss: 1.110663]
7 [D loss: 0.433635, acc: 61.72%, op_acc: 44.53%] [G loss: 1.119937]
8 [D loss: 0.411015, acc: 71.88%, op_acc: 46.88%] [G loss: 1.121502]
9 [D loss: 0.398081, acc: 71.09%, op_acc: 46.09%] [G loss: 1.145190]
10 [D loss: 0.399489, acc: 68.75%, op_acc: 46.09%] [G loss: 1.147179]
11 [D loss: 0.428585, acc: 67.97%, op_acc: 46.88%] [G loss: 0.948619]
12 [D loss: 0.479357, acc: 60.16%, op_acc: 44.53%] [G loss: 1.055347]
13 [D loss: 0.404078, acc: 71.88%, op_acc: 44.53%] [G loss: 1.046177]
14 [D loss: 0.390370, acc: 74.

In [19]:
num_classes = 10
(_, _), (X_test, y_test) = mnist.load_data()

# Rescale -1 to 1
X_test = (X_test.astype(np.float32) - 127.5) / 127.5
X_test = np.expand_dims(X_test, axis=3)
y_test = y_test.reshape(-1, 1)

y_test = to_categorical(y_test, num_classes=num_classes+1)
v_test = np.ones((y_test.shape[0], 1))
print(X_test.shape, y_test.shape,v_test.shape)

(10000, 28, 28, 1) (10000, 11) (10000, 1)


In [20]:
test_gan.discriminator.evaluate(X_test,[v_test, y_test])

  864/10000 [=>............................] - ETA: 1s

G:\Anaconda2\envs\tensorflow\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input2931") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
G:\Anaconda2\envs\tensorflow\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input2930") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


 9856/10000 [============================>.] - ETA: 0s

[0.23167262239456177,
 0.37263532958030698,
 0.090709915404021735,
 0.91180000000000005,
 0.97199999999999998]

In [21]:
test_gan.discriminator.metrics_names

['loss', 'dense_1_loss', 'dense_2_loss', 'dense_1_acc', 'dense_2_acc']

In [22]:
y_pred = test_gan.discriminator.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score

In [37]:
accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred[1], axis=1))

0.97199999999999998